## Demonstration of the User-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing user interactions, such as scroll length and read time, to identify users with similar behavior. 
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.



In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from utils.data_preprocessing import DataProcesser
from models.hybrid.most_popular_user_CF import MostPopularCollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data Import and EDA

In [2]:
dataProcesser = DataProcesser()
behaviors_df = dataProcesser.collaborative_filtering_preprocess()
train_df, test_df = dataProcesser.random_split(behaviors_df, test_ratio=0.3)
print(train_df.head())

shape: (5, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9776688    ┆ 2515395 ┆ 122.0          ┆ 100.0      │
│ 9790360    ┆ 1415869 ┆ 98.0           ┆ 100.0      │
│ 9782884    ┆ 2239705 ┆ 1.7649e10      ┆ 100.0      │
│ 9784591    ┆ 164545  ┆ 105.0          ┆ 100.0      │
│ 9776287    ┆ 2501949 ┆ 47.0           ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [3]:
recommender = MostPopularCollaborativeRecommender(train_df)
recommender.fit(scroll_weight=0.1, readtime_weight=1.0)

{2515395: [(1208989, np.float64(0.5107309792029082)),
  (615487, np.float64(0.47590841243925286)),
  (142466, np.float64(0.35170945302766343)),
  (1924667, np.float64(0.3228069839672243)),
  (1861121, np.float64(0.3213424045950555)),
  (1908663, np.float64(0.3211414490443739)),
  (1433894, np.float64(0.3211414490443739)),
  (1600415, np.float64(0.3211414490443738)),
  (554376, np.float64(0.3211414490443738)),
  (2413541, np.float64(0.3211414490443738))],
 1415869: [(1974712, np.float64(0.5956559596302026)),
  (2055085, np.float64(0.5667434013773784)),
  (613771, np.float64(0.5667434013773784)),
  (1796909, np.float64(0.5667434013773784)),
  (896126, np.float64(0.5667434013773784)),
  (1862214, np.float64(0.5667434013773784)),
  (1235986, np.float64(0.5667113710376316)),
  (2001710, np.float64(0.5662695349656678)),
  (2475648, np.float64(0.5660608462787339)),
  (1316072, np.float64(0.5655915483218293))],
 2239705: [(2239705, np.float64(1.0)),
  (567778, np.float64(1.0)),
  (2307527, np.

This first model just compares all artilces read by users when comparing users

In [4]:
binary_recommender = MostPopularCollaborativeRecommender(train_df, binary_model=True)
binary_recommender.fit()

{2515395: [(1924667, np.float64(0.3849001794597505)),
  (507868, np.float64(0.33333333333333326)),
  (193454, np.float64(0.33333333333333326)),
  (893005, np.float64(0.33333333333333326)),
  (1473633, np.float64(0.33333333333333326)),
  (1522656, np.float64(0.33333333333333326)),
  (1808796, np.float64(0.33333333333333326)),
  (1665036, np.float64(0.33333333333333326)),
  (2173365, np.float64(0.33333333333333326)),
  (2317541, np.float64(0.33333333333333326))],
 1415869: [(2533679, np.float64(0.3638034375544994)),
  (1247906, np.float64(0.34299717028501764)),
  (446844, np.float64(0.280056016805602)),
  (893005, np.float64(0.24253562503633297)),
  (916566, np.float64(0.24253562503633297)),
  (379929, np.float64(0.24253562503633297)),
  (83870, np.float64(0.24253562503633297)),
  (581491, np.float64(0.24253562503633297)),
  (1750171, np.float64(0.24253562503633297)),
  (2432499, np.float64(0.24253562503633297))],
 2239705: [(1667838, np.float64(1.0)),
  (1079121, np.float64(1.0)),
  (23

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model Presentation

### Article Recommendation

In [5]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9778351, 9771168, 9776147, 9771333, 9784444]
reccomended for user  620796 :  [9771224, 9771996, 9776967, 9772923, 9771113]
reccomended for user  1067393 :  [9771113, 9774789, 9671904, 9780849, 9785048]
reccomended for user  1726258 :  [9771224, 9771996, 9776967, 9772923, 9771113]
reccomended for user  17205 :  [9783657, 9774187, 9773486, 9779289, 9771948]


In [6]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9774142, 9783334, 9779860, 9778628, 9777856]
reccomended for user  620796 :  [9771686, 9783278, 9783334, 9779737, 9771168]
reccomended for user  1067393 :  [9771113, 9783752, 9769424, 9098807, 9776916]
reccomended for user  1726258 :  [9771224, 9774376, 9771113, 9773464, 9776041]
reccomended for user  17205 :  [9775562, 9777804, 9771846, 9772925, 9779674]


### Evaluation Scores

#### Without the Ability to Recommend Read Articles

The complex model only reccomending articles the user has not yet read

In [7]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=20, allow_read_articles=False)
results

{'MAP@K': np.float64(0.006153846153846154),
 'NDCG@K': np.float64(0.05601158526173388)}

The binary reccomender model only reccomending articles the user has not yet read

In [8]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=20, allow_read_articles=False)
results

{'MAP@K': np.float64(0.016923076923076926),
 'NDCG@K': np.float64(0.10213225754708145)}

#### With the Ability to Recommend Previously Read Articles

The complex model reccomending articles the user, even if they have read them before

In [9]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=20, allow_read_articles=True)
results

{'MAP@K': np.float64(0.006), 'NDCG@K': np.float64(0.05054836092642182)}

The binary reccomender model reccomending articles the user, even if they have read them before

In [10]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=20, allow_read_articles=True)
results

{'MAP@K': np.float64(0.012142857142857144),
 'NDCG@K': np.float64(0.07912753635049956)}

## Model Experimentation

In [11]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9772557, 9783824, 9774864, 9776916, 9776406, 9779615, 9789473, 9428643, 9774120, 9782315, 9774764, 9780020, 9772355, 9771460, 9538375, 9782092, 9771473, 9783509, 9772508, 9778657, 9776497, 9781875, 9788661, 9781878, 9776246, 9786618, 9778939, 9775484}
[9778351, 9771168, 9776147, 9771333, 9784444, 9772813, 9772300, 9782884, 9783057, 9776560, 9772095, 9772903, 9781998, 9774972, 9775983, 9778139, 9790784, 9777621, 9769917, 9780697, 9776041, 9774383, 9775647, 9790987, 9754241, 9785668, 9772475, 9771796, 9781389, 9772088, 9781991, 9771903, 9773210, 9789517, 9776855, 9771919, 9770327, 9771916, 9773282, 9776234, 9775562, 9790548, 9785475, 9775776, 9780195, 9782423, 9781086, 9779748, 9781476, 9786932, 9789810, 9787524, 9782092, 9779737, 9789065, 9787465, 9778168, 9784591, 9780280, 9776508, 9786378, 9774074, 9770082, 9775484, 9778804, 9782407, 9783213, 9774142, 9782517, 9773846, 9784856, 9774079, 9769504, 9785835, 9779289, 9783278, 9779860, 9773744, 9787176, 9784947, 9783850, 9779538, 9774363,

In [12]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9772557, 9783824, 9774864, 9776916, 9776406, 9779615, 9789473, 9428643, 9774120, 9782315, 9774764, 9780020, 9772355, 9771460, 9538375, 9782092, 9771473, 9783509, 9772508, 9778657, 9776497, 9781875, 9788661, 9781878, 9776246, 9786618, 9778939, 9775484}
[9778351, 9771168, 9776147, 9771333, 9784444, 9772813, 9772300, 9782884, 9783057, 9776560, 9772095, 9772903, 9781998, 9774972, 9775983, 9778139, 9790784, 9777621, 9780697, 9769917, 9776041, 9774383, 9775647, 9790987, 9754241, 9785668, 9772475, 9771796, 9781389, 9772088, 9781991, 9771903, 9773210, 9789517, 9771919, 9776855, 9770327, 9771916, 9773282, 9776234, 9775562, 9790548, 9785475, 9775776, 9780195, 9782423, 9781086, 9779748, 9781476, 9786932, 9789810, 9787524, 9782092, 9779737, 9789065, 9787465, 9778168, 9784591, 9780280, 9776508, 9786378, 9774074, 9770082, 9775484, 9778804, 9782407, 9783213, 9774142, 9782517, 9773846, 9784856, 9774079, 9769504, 9785835, 9779289, 9783278, 9779860, 9773744, 9787176, 9784947, 9783850, 9779538, 9774363,

In [13]:
from utils.evaluation import perform_model_evaluation
from utils.evaluation import append_model_metrics

matrics = perform_model_evaluation(recommender, test_df, k=5)
matrics

append_model_metrics(matrics, "most_popular_user_CF_hybrid")


### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [14]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "most_popular_user_CF_hybrid", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 13:16:29] [setup] RAM Tracking...
[codecarbon INFO @ 13:16:29] [setup] CPU Tracking...
[codecarbon WARNING @ 13:16:29] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon WARNING @ 13:16:31] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700H but we don't know it. Please contact us.
[codecarbon INFO @ 13:16:31] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 13:16:31] [setup] GPU Tracking...
[codecarbon INFO @ 13:16:31] No GPU found.
[codecarbon INFO @ 13:16:31] >>> Tracker's metadata:
[codecarbon INFO @ 13:16:31]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 13:16:31]   Python version: 3.11.9
[codecarbon INFO @ 13:16:31]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 13:16:31]   Available RAM : 15.731 GB
[codecarbon INFO @ 13:16:31]   CPU count: 20
[codecarbon INFO @ 13:16:31]   CPU model: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 13:16:31]   GPU count: None
[codecarbon INFO @ 13:16:31]   GPU model: None
[codecarbon INFO @ 13:16:34] Saving emissions data to file c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\demostrations\output\most_popular_us

{'fit': ({2515395: [(1208989, np.float64(0.4277956955624548)),
    (615487, np.float64(0.41045262682359085)),
    (1924667, np.float64(0.3630354690411697)),
    (1861121, np.float64(0.33419155686298585)),
    (1536665, np.float64(0.33337232131681327)),
    (1600415, np.float64(0.33337232131681327)),
    (16184, np.float64(0.33337232131681327)),
    (2342005, np.float64(0.33337232131681327)),
    (2017302, np.float64(0.33337232131681316)),
    (1101175, np.float64(0.33337232131681316))],
   1415869: [(1974712, np.float64(0.48088387538582866)),
    (1796909, np.float64(0.4410750719124338)),
    (613771, np.float64(0.4410750719124338)),
    (2055085, np.float64(0.4410750719124338)),
    (896126, np.float64(0.44107507191243367)),
    (1862214, np.float64(0.44107507191243367)),
    (1235986, np.float64(0.44101006251038033)),
    (2001710, np.float64(0.44059234865371155)),
    (1316072, np.float64(0.4401785322135162)),
    (1565321, np.float64(0.4271188432279036))],
   2239705: [(2239705, np